`otto` automatically tests functions and classes.  Defining classes should do more than change the namespace.

    %reload_ext XXX.otto
    
to install automatic testing.

In [9]:
    from unittest import *
    from doctest import *
    from ast import *
    from functools import wraps, partial
    from typing import Callable
    from dataclasses import dataclass, field
    from collections import Iterable
    from inspect import *
    from hypothesis import given, strategies, strategies as st, assume, HealthCheck, Verbosity, settings, find
    from behave import runner, configuration, parser
    from behave.runner import Context, ModelRunner
    __all__ = 'infer', 'INFER'
    
    INFER = True
    CURRENT = None
    def infer(object):
        spec = getfullargspec(object)
        annotations = {}#{'return': None}
        if not spec.args: return FunctionTestCase(object)
        annotations.update(spec.annotations) 
        
        main = __import__('__main__')
        for arg in spec.args:
            try:
                thing = eval(arg, vars(main))
            except NameError: continue
            if isinstance(thing, (type, Iterable)):
                annotations[arg] = thing
                
        if not spec.defaults:
            annotations = {
                str: st.from_type(object) if isinstance(object, type) and getattr(object, '__name__', '') != 'object'
                else object if isinstance(object, st.SearchStrategy)
                else st.one_of(list(map(st.just, object)))
                for str, object in annotations.items()
            }
#             del annotations['return']
            if annotations:
                return FunctionTestCase(given(**annotations)(object))


Allow iterable strategies, ghetto typed st

In [17]:
    @dataclass
    class Testing(object):
        shell: 'ip' = field(default_factory=get_ipython)
        tests = list()
            
        def post_run_cell(Testing):
            from types import ModuleType
            global INFER
            module, main = ModuleType('__main__'), __import__('__main__')
            tests = list()
            module.__test__ = getattr(main, '__test__', {})
            while Testing.tests:
                object = Testing.tests.pop(0)
                current = str(len(module.__test__))
                if isinstance(object, Str) and '>>> ' in object.s:
                    module.__test__[current] = object.s
                if isinstance(object, Str) and object.s.lstrip().startswith('Feature:'):
                    from behave import configuration, parser
                    from behave.runner import ModelRunner
                    tests.append(FunctionTestCase(ModelRunner(
                        config=configuration.Configuration(tuple()), 
                        features=[parser.parse_feature(object.s)]
                    ).run))
                if isinstance(object, ClassDef):    
                    object = getattr(main, object.name, None)
                    if issubclass(object, TestCase):
                        tests.append(object())
                    elif hasattr(object, 'runTest'):
                        def runTest(): return object.runTest(object)
                        tests.append(FunctionTestCase(wraps(object.runTest)(runTest)))
                if isinstance(object, FunctionDef):
                    object = getattr(main, object.name, None)
                    if not isinstance(object, partial):
                        try:
                            if INFER:
                                _test = infer(object)
                                _test and tests.append(_test)
                        except: ...
                        if getattr(object, '__doc__', ''):
                            module.__test__[object.__name__] = object
            if tests or module.__test__:
                tests.append(DocTestSuite(module, vars(main)))
                suite = TestSuite(tests)
                TextTestRunner().run(suite)
                
#             

In [11]:
    class DiscoverTests(NodeTransformer):
        def visit_star(self, node):
            Testing.tests += node,
            return node
        visit_Str = visit_ClassDef =  visit_FunctionDef = visit_star

In [12]:
    settings.register_profile('ip', settings(
        suppress_health_check=(HealthCheck.return_value,),
        verbosity=Verbosity.normal,))


In [5]:
    def load_ipython_extension(ip=get_ipython()):
        global testing
        testing = Testing(shell=get_ipython())
        ip.ast_transformers = remove_discover()+[DiscoverTests()]
        ip.events.register('post_run_cell', testing.post_run_cell)
        settings.load_profile('ip')

In [6]:
    def remove_discover(ip=get_ipython()):
        ip.ast_transformers = [_ for _ in ip.ast_transformers if not isinstance(_, DiscoverTests)]
        return ip.ast_transformers

In [7]:
    CURRENT = None
    def unload_ipython_extension(ip=get_ipython()):
        global CURRENT
        remove_discover()
        if CURRENT is not None:
            ip.events.unregister('post_run_cell', CURRENT.post_run_cell)

In [8]:
    if __name__ == '__main__':
        load_ipython_extension()
    

    !jupyter nbconvert --inplace --execute Untitled2.ipynb

    class context(Context):
        def __init__(Context):
            super().__init__(ModelRunner(config=configuration.Configuration(tuple())))

    def step_impl(context):
        print(context)
        pass